In [1]:
import torch
from data.data import *
from train import load_checkpoint
from cfg.configs import Config

nncfg = Config("nn_vls_dense.yaml")
model_dir = nncfg.config['model']['log_dir']
ds = nncfg.get_dataset()
%load_ext autoreload
%autoreload 2

transforming inputs


In [2]:
impcfg = Config("nn_vls_lg.yaml")

sirencfg = Config("nn_vls_siren.yaml")
rescfg = Config("nn_vls_res.yaml")
attncfg = Config("nn_vls_attn.yaml")
densecfg = Config("nn_vls_dense.yaml")

# mlp = load_checkpoint("log/mlp.pt", impcfg)
siren = load_checkpoint("log/siren.pt", sirencfg)
res = load_checkpoint(  "log/res.pt",   rescfg)
attn = load_checkpoint( "log/attn.pt",  attncfg)
dense = load_checkpoint("log/dense.pt", densecfg)

ds = sirencfg.get_dataset()

ckpts = [ siren, res, attn, dense]
models = [ckpt['model'] for ckpt in ckpts]
names = [ "siren", "res", "attn", "dense"]

transforming inputs


In [5]:
params = ds.inpp.get_params()
stder = params['steps'][0][1]
normer = params['steps'][1][1]

In [14]:
stats = np.array([stder.mean_, stder.scale_, normer.data_min_, normer.data_max_], dtype=np.float32)
stats

array([[147.92345  ],
       [219.87125  ],
       [ -1.809802 ],
       [  2.4927158]], dtype=float32)

In [15]:
num = np.array([4], dtype=np.int32)
with open("log/ts/stats.bin", 'wb') as f:
  f.write(num.tobytes())
  f.write(stats.tobytes())

In [3]:
example = torch.rand(2, 3)
for i in range(len(models)):
  traced_script_module = torch.jit.trace(models[i], example)
  traced_script_module.save(f"log/ts/{names[i]}.pt")

In [6]:
stand = ds.inpp.get_params()['steps'][0][1]
norm = ds.inpp.get_params()['steps'][1][1]

In [13]:
norm.get_params()

{'copy': True, 'feature_range': (0, 1)}

In [1]:
params = [stand.mean_, stand.scale_, norm.data_max_, norm.data_min_]

NameError: name 'stand' is not defined